### Install pre-requisite packages for the course

Please ensure that this cell is run, to install all pre-requisite packages needed for subsequent code examples.

In [1]:
!pip install python-dotenv
!pip install llama-index
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai
!pip install llama-index-llms-azure-openai
!pip install llama-index-embeddings-azure-openai
!pip install llama-index-agent-introspective
!pip install llama-index-program-openai
!pip install llama-index-readers-file
!pip install pydantic
!pip install llama-index-core
!pip install llama-index-utils-workflow
!pip install llama-index-tools-wikipedia
!pip install llama-index-readers-json
!pip install llama-index-readers-file
!pip install pymupdf
!pip install llama-index-tools-google
!pip install nbformat

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 54.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 32.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.8/431.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB

### 01.04. Setup LLM and Tools in LlamaIndex

#### Setup the LLM

In [2]:
#Using AzureOpenAI as LLM. Other LLM options are available in LlamaIndex
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()

#API key for AzureOpenAI. Replace with your own key
api_key = "4e4ab31800a64ae892cbb768fe28c0fc"
#Azure Endpoint URL. Replace with your own URL
azure_endpoint = "https://agentic-ai-course-kumaran.openai.azure.com/"

#Note : Function calling support only available in GPT-4+
#Create the LLM object
llm=AzureOpenAI(
    model="gpt-4o",
    deployment_name="gpt-4o",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version="2024-08-01-preview",
)

#### Set up Functions

In [3]:
#define a couple of functions
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y

### 01.05. Setup and execute an AI Agent

In [4]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.base import ReActAgent

#Create function tools 
sum_tool = FunctionTool.from_defaults(find_sum)
product_tool=FunctionTool.from_defaults(find_product)

#Create a pre-built ReAct agent
simple_agent = ReActAgent.from_tools(
                [sum_tool, product_tool],
                llm=llm,
                verbose=True #Set verbose for detailed logs
                )

#### Test the Agent

In [5]:
response = simple_agent.chat("What is 2 + 2? ")

> Running step 84755057-2ec0-44ae-92b2-6e09bfe1fecb. Step input: What is 2 + 2? 
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_sum
Action Input: {'x': 2, 'y': 2}
Observation: 4
> Running step 1fc4a770-3bfd-4077-8ce9-366e515c9834. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 2 + 2 is 4.


In [6]:
response = simple_agent.chat("Multiple 2 by 3 and then add 4")

> Running step 505befae-95b2-4240-9449-22dc9921bb38. Step input: Multiple 2 by 3 and then add 4
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_product
Action Input: {'x': 2, 'y': 3}
Observation: 6
> Running step e48d074a-a6a1-49fe-b728-d61f67e4ae4a. Step input: None
Thought: I have the product of 2 and 3, which is 6. Now I need to add 4 to this result.
Action: find_sum
Action Input: {'x': 6, 'y': 4}
Observation: 10
> Running step 36d714ce-1f3b-40a8-9342-3e019db3c447. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of multiplying 2 by 3 and then adding 4 is 10.
